In [1]:
import pandas as pd
import altair as alt
alt.renderers.set_embed_options(theme='dark')


RendererRegistry.enable('default')

In [2]:
def claim(claim_hash, cols_claim):
    try:
        df_claim = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"],
        )
    except:
        return pd.DataFrame(columns = cols_claim[claim_hash])
    if(len(df_claim.columns)==0):
        return pd.DataFrame(columns = cols_claim[claim_hash])
    return df_claim

In [3]:
p2_users = '07f87a6f-5bc6-49f3-a1bb-8e7d545ba95a'
p2_hourly = 'de5232e1-5d57-4232-900d-1d09d2c9438e'

In [4]:
cols_claim = {
    p2_users: ["DEPOSIT_AMOUNT","HR","NET_DEPOSITED_AMOUNT","SENDER","WITHDRAWN_AMOUNT","WITHDRAWN_AMOUNT_PHASE2"],
    p2_hourly: ['HR', 'WITH_AMOUNT', 'WITH_TX', 'WITH_USERS']
}

In [5]:
p2_users_df = claim(p2_users, cols_claim)
p2_hourly_df = claim(p2_hourly, cols_claim)

In [6]:
cols_dict = {
    "DEPOSIT_AMOUNT":"Amount deposited",
    "DEPOSIT_TXS":"N° of deposit txs",
    "DEP_USERS":"N° of depositing users",
    "HR":"Time",
    "NET_AMOUNT":"Net amount deposited",
    "TOT_TXS":"N° of txs",
    "TOT_USERS":"N° of users",
    "WITH_AMOUNT":"Withdrawn amount",
    "WITH_TXS":"N° of withdraw txs",
    "WITH_USERS":"N° of withdrawing users",
    'AMOUNT':'Amount deposited (UST)',
    'AVG_BALANCE_USD':'Average balance in the last 30 days (UST)',
    'N_TXS':'N° of txs',
    'SENDER':'User address',
    'N_USERS':'N° of users',
    'BUCKET':'Deposit range',
    'bucket_name':'Deposit bucket',
    'cumsum_ust':'Cumulative UST amount',
    'MIN_DATE':'Date wallet creation',
    'ADDRESS_COUNT':'Number of wallets',
    'PERC_WITHDRAWN':'% withdrawn of the deposited amount',
    'net_ust': 'Amount UST'
}

In [7]:
#Total withdrawn phase 2 UST
tot_with_ust = p2_users_df['WITHDRAWN_AMOUNT_PHASE2'].sum()
tot_with_ust

871030.5994549999

In [8]:
#Total net dep phase 1 UST
tot_ust_p1 = p2_users_df['NET_DEPOSITED_AMOUNT'].sum()
tot_ust_p1

4394255.454423999

In [9]:
#Total left UST
tot_net_ust = tot_ust_p1 - tot_with_ust
tot_net_ust

3523224.8549689995

In [10]:
#% withdrawn
perc_with_p2 = tot_with_ust/tot_ust_p1 * 100
perc_with_p2

19.82202920356106

In [11]:
#current price
curr_price = tot_net_ust/70000000
curr_price

0.05033178364241428

In [12]:
#% users with
p_users_with_p2 = (1-p2_users_df['WITHDRAWN_AMOUNT_PHASE2'].isna().sum()/len(p2_users_df))*100
p_users_with_p2

5.04246284501062

In [13]:
#Floor price
p2_users_df['max_with_hour'] = '2022-02-03 10:00:00.000'
possible_with = p2_users_df[p2_users_df['WITHDRAWN_AMOUNT_PHASE2'].isna()]['NET_DEPOSITED_AMOUNT']

In [14]:
p2_users_df.head()

,DEPOSIT_AMOUNT,HR,NET_DEPOSITED_AMOUNT,SENDER,WITHDRAWN_AMOUNT,WITHDRAWN_AMOUNT_PHASE2,max_with_hour
0,1000.0,2022-02-02 09:00:00.000,400.0,terra1v6ujue0uqd9nzgtqvuhwncaqd9q6hacvsfur7x,600.0,600.0,2022-02-03 10:00:00.000
1,101.0,2022-02-01 11:00:00.000,100.0,terra17js7wjaq8w80khztfa9ydz0etdgzy4l7t8yfsa,1.0,1.0,2022-02-03 10:00:00.000
2,99500.0,2022-02-01 20:00:00.000,0.0,terra1frsd6nayz25encp6w6m8z00g5ue4j6fp4q0pza,99500.0,99500.0,2022-02-03 10:00:00.000
3,185.0,2022-02-01 12:00:00.000,35.0,terra1lcrwyxn8apvxu9jdvu6jk6myeey0z64e22du4k,150.0,150.0,2022-02-03 10:00:00.000
4,100.0,2022-02-01 22:00:00.000,0.0,terra1nrg8s0wtd2z0xyl26u669qh87qcn2ql0wepvhj,100.0,100.0,2022-02-03 10:00:00.000


In [15]:
#UST left to be withdrawn
df = p2_users_df[p2_users_df['HR'].notna()]
delta = pd.to_datetime(df['max_with_hour']) - pd.to_datetime(df['HR'])
df['hours_til_end_p2'] = delta.dt.days * 24 + delta.dt.seconds/3600
df['left_to_with'] = 24/df['hours_til_end_p2'] * df['NET_DEPOSITED_AMOUNT']
left_to_with = df['left_to_with'].sum()
left_to_with

C:\Users\aincerti\AppData\Local\Temp\ipykernel_16196\2705070969.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hours_til_end_p2'] = delta.dt.days * 24 + delta.dt.seconds/3600
C:\Users\aincerti\AppData\Local\Temp\ipykernel_16196\2705070969.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['left_to_with'] = 24/df['hours_til_end_p2'] * df['NET_DEPOSITED_AMOUNT']


66673.06104211377

In [16]:
#Floor price
floor_price = (tot_net_ust - left_to_with)/70000000
floor_price

0.04937931134181266

In [17]:
p2_hourly_df['HR'] = '2022/' + p2_hourly_df['HR'] + ':00'
p2_hourly_df.head()

,HR,WITH_AMOUNT,WITH_TX,WITH_USERS
0,2022/02/02 11:00:00,1070.000000,2,2
1,2022/02/02 09:00:00,1150.000000,4,4
2,2022/02/01 20:00:00,104711.479309,5,5
3,2022/02/01 12:00:00,857.670794,3,3
4,2022/02/01 10:00:00,19312.000000,3,3


In [18]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_bar().encode(
    x=cols_dict['HR']+':T',
    y=cols_dict['WITH_AMOUNT']+":Q",
    tooltip=[cols_dict['HR']+':T',cols_dict['WITH_AMOUNT']+":Q"]
).configure_mark(
    color='#fab0ba'
).properties(width=700).configure_axisX(
    labelAngle=30
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [19]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_bar().encode(
    x=cols_dict['HR']+':T',
    y=cols_dict['WITH_USERS']+":Q",
    tooltip=[cols_dict['HR']+':T',cols_dict['WITH_USERS']+":Q"]
).configure_mark(
    color='#fab0ba'
).properties(width=700).configure_axisX(
    labelAngle=30
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [32]:
with_users_df = p2_users_df[p2_users_df['HR'].notna()&p2_users_df['NET_DEPOSITED_AMOUNT']>0]
with_users_df['perc_withdrawn'] = with_users_df[with_users_df['HR'].notna()]['WITHDRAWN_AMOUNT']/with_users_df['NET_DEPOSITED_AMOUNT']*100
with_users_df['perc_withdrawn_cat'] = (with_users_df['perc_withdrawn']/100).apply(lambda x: int(x))
df2 = with_users_df.groupby('perc_withdrawn_cat').SENDER.count()

C:\Users\aincerti\AppData\Local\Temp\ipykernel_16196\1346942249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['perc_withdrawn'] = with_users_df[with_users_df['HR'].notna()]['WITHDRAWN_AMOUNT']/with_users_df['NET_DEPOSITED_AMOUNT']*100
C:\Users\aincerti\AppData\Local\Temp\ipykernel_16196\1346942249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['perc_withdrawn_cat'] = (with_users_df['perc_withdrawn']/100).apply(lambda x: int(x))


In [27]:
perc_cat = list(range(10))
cat = pd.DataFrame([0]*10,perc_cat)
df3 = cat.join(df2,how='outer')
df3.index =  ['0%-10%','10%-20%','20%-30%','30%-40%','40%-50%','50%-60%','60%-70%','70%-80%','80%-90%','90%-100%']
df3 = df3.SENDER.fillna(0).reset_index()
df3=df3.rename(columns={'index':'PERC_WITHDRAWN','SENDER':'TOT_USERS'})

In [28]:
alt.Chart(df3.sort_values(by='PERC_WITHDRAWN').rename(columns=cols_dict)).mark_bar().encode(
    y=alt.X(cols_dict['PERC_WITHDRAWN']+":N", sort=alt.EncodingSortField(order='ascending')),
    x=cols_dict['TOT_USERS']+":Q",
    tooltip=[cols_dict['PERC_WITHDRAWN']+":N",cols_dict['TOT_USERS']+":Q"]
).configure_mark(
    color='#F1705F'
).properties(height=300).configure_axisX(
    labelAngle=0
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [29]:
p2_hourly_df = p2_hourly_df.sort_values(by='HR')
p2_hourly_df['cumsum_with'] = p2_hourly_df.sort_values(by='HR').WITH_AMOUNT.cumsum()
p2_hourly_df['net_ust'] = tot_ust_p1 - p2_hourly_df['cumsum_with']
p2_hourly_df.head()

,HR,WITH_AMOUNT,WITH_TX,WITH_USERS,cumsum_with,net_ust
22,2022/02/01 07:00:00,501183.642552,11,11,501183.642552,3.893072e+06
14,2022/02/01 08:00:00,5923.464239,3,3,507107.106791,3.887148e+06
27,2022/02/01 09:00:00,1250.000000,2,2,508357.106791,3.885898e+06
4,2022/02/01 10:00:00,19312.000000,3,3,527669.106791,3.866586e+06
11,2022/02/01 11:00:00,24691.000000,7,7,552360.106791,3.841895e+06


In [30]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_line(point=True).encode(
    x=alt.X(cols_dict['HR']+':T', sort=alt.EncodingSortField(order='ascending')),
    y=cols_dict['net_ust']+":Q",
    tooltip=[alt.Tooltip(cols_dict['HR']+':T', format='%Y-%m-%d %H:%M'), alt.Tooltip(cols_dict['net_ust']+":Q")]
).configure_mark(
    color='#F1705F'
).properties(width=700).configure_axisX(
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [71]:
with_users_df['DEP_CAT'] = (with_users_df['NET_DEPOSITED_AMOUNT']/1000).apply(int)
df = with_users_df.groupby(['DEP_CAT','perc_withdrawn_cat']).SENDER.count()
df = df.reset_index()
df

C:\Users\aincerti\AppData\Local\Temp\ipykernel_16196\875801285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['DEP_CAT'] = (with_users_df['NET_DEPOSITED_AMOUNT']/1000).apply(int)


,DEP_CAT,perc_withdrawn_cat,SENDER
0,0,0,9
1,0,1,5
2,0,2,1
3,0,3,2
4,0,4,2
5,0,5,1
6,0,8,1
7,1,0,6
8,1,1,2
9,2,0,1


In [72]:
alt.Chart(df).mark_rect().encode(
    x=alt.X('perc_withdrawn_cat:O', sort=alt.EncodingSortField(order='ascending')),
    y=alt.Y('DEP_CAT:O', sort=alt.EncodingSortField(order='descending')),
    color='SENDER:Q'
)

alt.Chart(...)

In [94]:
import random
df1 = pd.DataFrame([list(range(0,100,5))]).T
df2 = pd.DataFrame([list(range(0,1000,50))]).T
heatmap_val = df2.merge(df1, how='cross')
heatmap_val.columns = ['DEP_CAT','perc_withdrawn_cat']
df = heatmap_val.merge(df, on=['perc_withdrawn_cat', 'DEP_CAT'], how='left').fillna(0)
df['SENDER'] = df['SENDER'].apply(lambda v: random.random() * 1000)
df = df.sort_values(by=['perc_withdrawn_cat', 'DEP_CAT'], ascending=[True,False])

In [95]:
df = df.sort_values(by=['perc_withdrawn_cat', 'DEP_CAT'], ascending=[True,False])

In [96]:
alt.Chart(df).mark_rect().encode(
    x=alt.X('perc_withdrawn_cat:O', sort=alt.EncodingSortField(order='ascending')),
    y=alt.Y('DEP_CAT:O', sort=alt.EncodingSortField(order='ascending')),
    color=alt.Color('SENDER:Q',
            scale=alt.Scale(scheme='redpurple')),
    tooltip=['perc_withdrawn_cat:O','DEP_CAT:O']
)

alt.Chart(...)